In [2]:
import time

notebook_start_time = time.time()

# Set up environment

In [3]:
import sys
from pathlib import Path


def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False


def clone_repository() -> None:
    !git clone https://github.com/chinadupaya/art-recommendations.git
    %cd art-recommendations/


def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml


if is_google_colab():
    clone_repository()
    install_dependencies()

    root_dir = str(Path().absolute())
    print("⛳️ Google Colab environment")
else:
    root_dir = str(Path().absolute().parent)
    print("⛳️ Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    print(f"Adding the following directory to the PYTHONPATH: {root_dir}")
    sys.path.append(root_dir)

⛳️ Local environment
Adding the following directory to the PYTHONPATH: /Users/annysiadupaya/Documents/code/art-recommendations


# 👩🏻‍🔬 Feature pipeline: Computing features

# Imports

In [4]:
%load_ext autoreload
%autoreload 2
# %reload_ext autoreload

import warnings
from pprint import pprint

import polars as pl
import torch
from loguru import logger
from sentence_transformers import SentenceTransformer

warnings.filterwarnings("ignore")

from recsys import hopsworks_integration
from recsys.config import settings
from recsys.features.artworks import (
    compute_features_artworks,
    generate_embeddings_for_dataframe,
)
from recsys.features.users import DatasetSampler, compute_features_users
from recsys.features.interaction import generate_interaction_data
from recsys.features.ranking import compute_ranking_dataset
from recsys.features.ranking3 import compute_ranking_dataset2
from recsys.features.transactions import compute_features_transactions
from recsys.hopsworks_integration import feature_store

/Users/annysiadupaya/Documents/code/art-recommendations/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Constants

The most important one is the dataset size.

Choosing a different dataset size will impact the time it takes to run everything and the quality of the final models. We suggest using a small dataset size when running this the first time.

Suported user dataset sizes:

In [6]:
DatasetSampler.get_supported_sizes()

{<UserDatasetSize.LARGE: 'LARGE'>: 50000,
 <UserDatasetSize.MEDIUM: 'MEDIUM'>: 5000,
 <UserDatasetSize.SMALL: 'SMALL'>: 1000}

## <span style="color:#ff5f27">🔮 Connect to Hopsworks Feature Store </span>

In [7]:
project, fs = hopsworks_integration.get_feature_store()

2024-12-22 20:22:37.298 | INFO     | recsys.hopsworks_integration.feature_store:get_feature_store:15 - Loging to Hopsworks using HOPSWORKS_API_KEY env var.


2024-12-22 20:22:37,298 INFO: Initializing external client
2024-12-22 20:22:37,299 INFO: Base URL: https://c.app.hopsworks.ai:443
2024-12-22 20:22:38,863 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1197208


In [8]:
artworks_df =  pl.read_csv("../data/artworks_info.csv")
artworks_df.shape

(18538, 8)

The artworks DataFrame looks as follows

In [9]:
artworks_df.head(3)

id,title,category,thumbnail_link,artists_link,genes_link,similar_link,description
str,str,str,str,str,str,str,str
"""4d8b92eb4eb68a1b2c000968""","""Der Kuss (The Kiss)""","""Painting""","""https://d32dm0rphc51dk.cloudfr…","""https://api.artsy.net/api/arti…","""https://api.artsy.net/api/gene…","""https://api.artsy.net/api/artw…","""The image is rich in golds and…"
"""4d8b92ee4eb68a1b2c0009ab""","""The Third of May""","""Painting""","""https://d32dm0rphc51dk.cloudfr…","""https://api.artsy.net/api/arti…","""https://api.artsy.net/api/gene…","""https://api.artsy.net/api/artw…","""The image features a somber pa…"
"""4d8b93394eb68a1b2c0010fa""","""The Company of Frans Banning C…","""Painting""","""https://d32dm0rphc51dk.cloudfr…","""https://api.artsy.net/api/arti…","""https://api.artsy.net/api/gene…","""https://api.artsy.net/api/artw…","""The image features a rich pale…"


In [10]:
artworks_df.null_count()

id,title,category,thumbnail_link,artists_link,genes_link,similar_link,description
u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0


## Artworks Feature Engineering

In [11]:
artworks_df = compute_features_artworks(artworks_df)
artworks_df.shape


(18538, 5)

In [12]:
artworks_df.head()

artwork_id,title,category,thumbnail_link,description
str,str,str,str,str
"""4d8b92eb4eb68a1b2c000968""","""Der Kuss (The Kiss)""","""Painting""","""https://d32dm0rphc51dk.cloudfr…","""The image is rich in golds and…"
"""4d8b92ee4eb68a1b2c0009ab""","""The Third of May""","""Painting""","""https://d32dm0rphc51dk.cloudfr…","""The image features a somber pa…"
"""4d8b93394eb68a1b2c0010fa""","""The Company of Frans Banning C…","""Painting""","""https://d32dm0rphc51dk.cloudfr…","""The image features a rich pale…"
"""4d8b937c4eb68a1b2c001722""","""Mona Lisa""","""Painting""","""https://d32dm0rphc51dk.cloudfr…","""The image is characterized by …"
"""4d8b93b04eb68a1b2c001b9d""","""Luncheon on the Grass (Le Déje…","""Painting""","""https://d32dm0rphc51dk.cloudfr…","""The image features a palette o…"


### Create embeddings from the articles description

In [13]:
for i, desc in enumerate(artworks_df["description"].head(n=3)):
    logger.info(f"Item {i+1}:\n{desc}")

2024-12-22 20:22:40.716 | INFO     | __main__:<module>:2 - Item 1:
The image is rich in golds and soft pastels, creating a warm, harmonious atmosphere. The mood is intimate and tender, evoking feelings of love and closeness. The aesthetic features intricate patterns and floral motifs, reminiscent of the Art Nouveau style. Overall, it radiates a sense of tranquility and beauty.
2024-12-22 20:22:40.716 | INFO     | __main__:<module>:2 - Item 2:
The image features a somber palette dominated by dark tones and stark contrasts, highlighting the brutality of the scene. The mood conveys a sense of despair and urgency, evoking empathy for the victims. The dramatic lighting, particularly the lantern's glow, creates a tension between hope and tragedy. Overall, the aesthetic is visceral and haunting, capturing the raw emotion of human conflict.
2024-12-22 20:22:40.716 | INFO     | __main__:<module>:2 - Item 3:
The image features a rich palette of dark hues, contrasted by bright highlights, creatin

In [14]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
logger.info(
    f"Loading '{settings.FEATURES_EMBEDDING_MODEL_ID}' embedding model to {device=}"
)

# Load the embedding model from SentenceTransformer's model registry.
model = SentenceTransformer(settings.FEATURES_EMBEDDING_MODEL_ID, device=device)

2024-12-22 20:22:40.746 | INFO     | __main__:<module>:8 - Loading 'all-MiniLM-L6-v2' embedding model to device='mps'


2024-12-22 20:22:40,747 INFO: Load pretrained SentenceTransformer: all-MiniLM-L6-v2


In [15]:
embedding = generate_embeddings_for_dataframe(
    artworks_df.head(1), "description", model, batch_size=128
)['embeddings']
embedding

Generating embeddings: 100%|██████████| 1/1 [00:00<00:00,  8.37it/s]


embeddings
list[f64]
"[-0.008282, 0.030209, … -0.003906]"


In [16]:
type(artworks_df)

polars.dataframe.frame.DataFrame

In [17]:
artworks_df = artworks_df.with_columns(
    pl.lit(embedding[0].to_list()).alias("embeddings")
)
artworks_df.head(3)

artwork_id,title,category,thumbnail_link,description,embeddings
str,str,str,str,str,list[f64]
"""4d8b92eb4eb68a1b2c000968""","""Der Kuss (The Kiss)""","""Painting""","""https://d32dm0rphc51dk.cloudfr…","""The image is rich in golds and…","[-0.008282, 0.030209, … -0.003906]"
"""4d8b92ee4eb68a1b2c0009ab""","""The Third of May""","""Painting""","""https://d32dm0rphc51dk.cloudfr…","""The image features a somber pa…","[-0.008282, 0.030209, … -0.003906]"
"""4d8b93394eb68a1b2c0010fa""","""The Company of Frans Banning C…","""Painting""","""https://d32dm0rphc51dk.cloudfr…","""The image features a rich pale…","[-0.008282, 0.030209, … -0.003906]"


In [18]:
artworks_df = generate_embeddings_for_dataframe(
    artworks_df, "description", model, batch_size=128
)  # Reduce batch size if getting OOM errors.

Generating embeddings: 100%|██████████| 18538/18538 [01:30<00:00, 205.10it/s]


For each artwork description, we have a numerical vector which we can feed to a model, opposite to a string containing the description of an object.

In [19]:
artworks_df[["description", "embeddings"]].head(3)

description,embeddings
str,list[f64]
"""The image is rich in golds and…","[-0.008282, 0.030209, … -0.003906]"
"""The image features a somber pa…","[0.058479, 0.083046, … 0.008479]"
"""The image features a rich pale…","[0.012057, 0.037521, … 0.004547]"


## Looking at image links

In [20]:
artworks_df["thumbnail_link"][0]

'https://d32dm0rphc51dk.cloudfront.net/NOpIAwQa-3r51Cg9qXKbfA/medium.jpg'

In [21]:
from IPython.display import HTML, display

image_urls = artworks_df["thumbnail_link"].tail(12).to_list()
grid_html = '<div style="display: grid; grid-template-columns: repeat(6, 1fr); gap: 10px; max-width: 900px;">'

for url in image_urls:
    grid_html += f'<img src="{url}" style="width: 100%; height: auto;">'

grid_html += "</div>"

display(HTML(grid_html))


# Users Data

In [25]:
users_df = pl.read_csv("../data/updated_user_details.csv")
users_df.shape


(54000, 5)

The users DataFrame looks as follows:

In [26]:
users_df.head(3)

user_id,literal,age,gender,preference
str,str,i64,str,str
"""69ff63b1-4803-49c9-8585-3b0f47…","""[Abbaye du Gard pres d' Abbevi…",42,"""Male""","""Impressionism"""
"""81cd5866-14fb-4968-80f2-accf84…","""[Italian Landscape, Sunset (So…",48,"""Female""","""Impressionism"""
"""83e00911-6b5a-44d3-9edb-3c02a2…","""[The Church of Souain, The Sei…",56,"""Female""","""Impressionism"""


Check for nulls

In [27]:
users_df.null_count()

user_id,literal,age,gender,preference
u32,u32,u32,u32,u32
0,0,0,0,0


# Users feature engineering

In [28]:
users_df = compute_features_users(users_df, drop_null_age=True)
users_df.shape

(54000, 6)

# Transactions Data
These are the artworks a user likes

In [29]:
transactions_df = pl.read_csv("../data/transaction-data.csv")
transactions_df.shape

(306100, 4)

Transactions DataFrame looks as follows

In [30]:
transactions_df.head(3)

transaction_id,user_id,artwork_id,thumbnail_link
str,str,str,str
"""f9f34e02-f51d-42ff-ada2-928728…","""636d1fa7-a3c2-4fe6-b278-265d29…","""516cbb4a0f8b7853440003fe""","""https://d32dm0rphc51dk.cloudfr…"
"""7f8ba386-bfe0-4c0a-b71e-c45c9b…","""8cea0e10-c960-44e9-b404-bc3047…","""516cbfd89ad2d38886000142""","""https://d32dm0rphc51dk.cloudfr…"
"""a2f1abf6-b58b-4f4a-a0ba-ee2fba…","""5ab9bc35-c1e5-4d99-bddd-499653…","""515b45c0223afa29bd000948""","""https://d32dm0rphc51dk.cloudfr…"


In [31]:
from datetime import datetime
t_dat = datetime.today()
# t_dat = datetime.today().strftime('%Y-%m-%d')
transactions_df = (
    transactions_df.with_columns(t_dat=pl.lit(t_dat))
)

transactions_df.head(3)

transaction_id,user_id,artwork_id,thumbnail_link,t_dat
str,str,str,str,datetime[μs]
"""f9f34e02-f51d-42ff-ada2-928728…","""636d1fa7-a3c2-4fe6-b278-265d29…","""516cbb4a0f8b7853440003fe""","""https://d32dm0rphc51dk.cloudfr…",2024-12-22 20:24:27.784716
"""7f8ba386-bfe0-4c0a-b71e-c45c9b…","""8cea0e10-c960-44e9-b404-bc3047…","""516cbfd89ad2d38886000142""","""https://d32dm0rphc51dk.cloudfr…",2024-12-22 20:24:27.784716
"""a2f1abf6-b58b-4f4a-a0ba-ee2fba…","""5ab9bc35-c1e5-4d99-bddd-499653…","""515b45c0223afa29bd000948""","""https://d32dm0rphc51dk.cloudfr…",2024-12-22 20:24:27.784716


# Transactions Feature Engineering

In [32]:
transactions_df = compute_features_transactions(transactions_df)
transactions_df.shape

(306100, 9)

In [33]:
transactions_df.head(3)

transaction_id,user_id,artwork_id,thumbnail_link,t_dat,year,month,day,day_of_week
str,str,str,str,i64,i32,i8,i8,i8
"""f9f34e02-f51d-42ff-ada2-928728…","""636d1fa7-a3c2-4fe6-b278-265d29…","""516cbb4a0f8b7853440003fe""","""https://d32dm0rphc51dk.cloudfr…",1734899067,2024,12,22,7
"""7f8ba386-bfe0-4c0a-b71e-c45c9b…","""8cea0e10-c960-44e9-b404-bc3047…","""516cbfd89ad2d38886000142""","""https://d32dm0rphc51dk.cloudfr…",1734899067,2024,12,22,7
"""a2f1abf6-b58b-4f4a-a0ba-ee2fba…","""5ab9bc35-c1e5-4d99-bddd-499653…","""515b45c0223afa29bd000948""","""https://d32dm0rphc51dk.cloudfr…",1734899067,2024,12,22,7


We don't want to work with ~300k transactions in these series, as everything will take too much time to run. Thus, we create a subset of the original dataset by randomly sampling from the users' datasets and taking only their transactions.

In [34]:
sampler = DatasetSampler(size=settings.USER_DATA_SIZE)
dataset_subset = sampler.sample(
    users_df=users_df, transations_df=transactions_df
)
users_df = dataset_subset["users"]
transactions_df = dataset_subset["transactions"]

2024-12-22 20:24:29.552 | INFO     | recsys.features.users:sample:29 - Sampling 1000 users.
2024-12-22 20:24:29.553 | INFO     | recsys.features.users:sample:32 - Number of transactions for all the users: 306100
2024-12-22 20:24:29.562 | INFO     | recsys.features.users:sample:38 - Number of transactions for the 1000 sampled users: 5606


In [35]:
transactions_df.shape

(5606, 9)

Some of the remaining customers

In [36]:
for user_id in transactions_df["user_id"].unique().head(10):
    logger.info(f"Logging user ID: {user_id}")

2024-12-22 20:24:29.626 | INFO     | __main__:<module>:2 - Logging user ID: 676c7c59-c2fb-4840-985a-b177c8cc8310
2024-12-22 20:24:29.626 | INFO     | __main__:<module>:2 - Logging user ID: f19b25fa-e9ca-4c56-9066-229ecb2c305f
2024-12-22 20:24:29.627 | INFO     | __main__:<module>:2 - Logging user ID: 01b29e84-3e35-44d1-8e59-ab8fa6cbe205
2024-12-22 20:24:29.627 | INFO     | __main__:<module>:2 - Logging user ID: 2dfabf75-62f8-4fbe-a7e9-a09fd8357d4f
2024-12-22 20:24:29.627 | INFO     | __main__:<module>:2 - Logging user ID: 97fe2438-f741-420e-bb76-57d9a80b5d6f
2024-12-22 20:24:29.627 | INFO     | __main__:<module>:2 - Logging user ID: 3d692dbf-4530-473b-8345-6890b063dfcc
2024-12-22 20:24:29.627 | INFO     | __main__:<module>:2 - Logging user ID: 560e4f40-fe88-4085-b40b-fa29a34bc59b
2024-12-22 20:24:29.628 | INFO     | __main__:<module>:2 - Logging user ID: 663cbad7-47d2-4389-9a89-c470bb730f36
2024-12-22 20:24:29.628 | INFO     | __main__:<module>:2 - Logging user ID: c44d00e7-72b2-447b-a

# 🤳🏻 Interaction data

To train our models, we need more than just the transactions DataFrame. We need positive samples that signal whether a user clicked or liked an item, but we also need negative samples that signal no interactions between a user and an artwork.

In [37]:
interaction_df = generate_interaction_data(transactions_df)
interaction_df.shape

Processing user chunks: 100%|██████████| 1/1 [00:00<00:00,  1.05it/s]


(93520, 5)

The interaction features look as follows:

In [38]:
interaction_df.head()

t_dat,user_id,artwork_id,interaction_score,prev_artwork_id
i64,str,str,i64,str
1364099067,"""00306dfa-3c17-4709-b9f4-834502…","""4fcf9efbcaabe500010018a7""",0,"""START"""
1364099067,"""00306dfa-3c17-4709-b9f4-834502…","""515b98f7ea092ff579000389""",0,"""4fcf9efbcaabe500010018a7"""
1364099067,"""00306dfa-3c17-4709-b9f4-834502…","""515cc3d8b5907bf7e800026f""",0,"""515b98f7ea092ff579000389"""
1374899067,"""00306dfa-3c17-4709-b9f4-834502…","""515b98f7ea092ff579000389""",0,"""515cc3d8b5907bf7e800026f"""
1403699067,"""00306dfa-3c17-4709-b9f4-834502…","""515b44b89562c86fb800232a""",0,"""515b98f7ea092ff579000389"""


Let's take a look at the interaction score distribution

In [39]:
interaction_df.group_by("interaction_score").agg(
    pl.count("interaction_score").alias("total_interactions")
)

interaction_score,total_interactions
i64,u32
0,74193
1,13721
2,5606


Here is what each score means:
- `0` : No interaction between a user and an artwork
- `1` : A user clicked an artwork
- `2` : A cuser liked an item

# <span style="color:#ff5f27">🪄 Create Hopsworks Feature Groups </span>

A [feature group](https://docs.hopsworks.ai/feature-store-api/latest/generated/feature_group/) can be seen as a collection of conceptually related features.

To create a feature group you need to give it a name and specify a primary key. It is also best practice to provide a description of the contents of the feature group.

#### Users

We set `online_enabled=True` to enable low-latency access to the data from the inference pipeline for real-time predictions. 

A full list of arguments can be found in the [documentation](https://docs.hopsworks.ai/feature-store-api/latest/generated/api/feature_store_api/#create_feature_group).

In [44]:
logger.info("Uploading 'users' Feature Group to Hopsworks.")
users_fg = feature_store.create_users_feature_group(
    fs, df=users_df, online_enabled=True
)

logger.info("✅ Uploaded 'users' Feature Group to Hopsworks!")

2024-12-22 20:27:10.783 | INFO     | __main__:<module>:1 - Uploading 'users' Feature Group to Hopsworks.
Uploading Dataframe: 100.00% |██████████| Rows 1000/1000 | Elapsed Time: 00:04 | Remaining Time: 00:00


Launching job: users_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1197208/jobs/named/users_1_offline_fg_materialization/executions
2024-12-22 20:27:32,189 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2024-12-22 20:27:35,400 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2024-12-22 20:29:12,046 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2024-12-22 20:29:12,213 INFO: Waiting for log aggregation to finish.
2024-12-22 20:29:21,133 INFO: Execution finished successfully.


2024-12-22 20:29:24.281 | INFO     | __main__:<module>:6 - ✅ Uploaded 'users' Feature Group to Hopsworks!


#### Artworks

Let's do the same thing for the rest of the data frames

In [ ]:
logger.info("Uploading 'artworks' Feature Group to Hopsworks.")
artworks_fg = feature_store.create_artworks_feature_group(
    fs,
    df=artworks_df,
    artworks_description_embedding_dim=model.get_sentence_embedding_dimension(),
    online_enabled=True,
)
logger.info("✅ Uploaded 'artworks' Feature Group to Hopsworks!")


2024-12-22 20:42:06.954 | INFO     | __main__:<module>:1 - Uploading 'artworks' Feature Group to Hopsworks.
%6|1734896571.349|FAIL|rdkafka#producer-3| [thrd:ssl://51.161.81.188:9093/bootstrap]: ssl://51.161.81.188:9093/1: Disconnected (after 49998ms in state UP, 1 identical error(s) suppressed)
%6|1734896572.089|FAIL|rdkafka#producer-3| [thrd:ssl://51.161.81.208:9093/bootstrap]: ssl://51.161.81.208:9093/2: Disconnected (after 49997ms in state UP, 1 identical error(s) suppressed)
%6|1734896622.222|FAIL|rdkafka#producer-3| [thrd:ssl://51.161.80.189:9093/bootstrap]: ssl://51.161.80.189:9093/0: Disconnected (after 100214ms in state UP, 1 identical error(s) suppressed)
%6|1734896675.906|FAIL|rdkafka#producer-3| [thrd:ssl://51.161.80.189:9093/bootstrap]: ssl://51.161.80.189:9093/0: Disconnected (after 50658ms in state UP, 1 identical error(s) suppressed)
%6|1734896676.675|FAIL|rdkafka#producer-3| [thrd:ssl://51.161.81.208:9093/bootstrap]: ssl://51.161.81.208:9093/2: Disconnected (after 50033

#### Transactions

In [45]:
logger.info("Uploading 'transactions' Feature Group to Hopsworks.")
trans_fg = feature_store.create_transactions_feature_group(
    fs=fs, df=transactions_df, online_enabled=True
)
logger.info("✅ Uploaded 'transactions' Feature Group to Hopsworks!")

2024-12-22 20:35:11.310 | INFO     | __main__:<module>:1 - Uploading 'transactions' Feature Group to Hopsworks.


ERROR! Session/line number was not unique in database. History logging moved to new session 70


Uploading Dataframe: 100.00% |██████████| Rows 5606/5606 | Elapsed Time: 00:05 | Remaining Time: 00:00


Launching job: transactions_1_offline_fg_materialization


%6|1734896124.236|FAIL|rdkafka#producer-3| [thrd:ssl://51.161.81.208:9093/bootstrap]: ssl://51.161.81.208:9093/2: Disconnected (after 150981ms in state UP)


Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1197208/jobs/named/transactions_1_offline_fg_materialization/executions
2024-12-22 20:35:32,808 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2024-12-22 20:35:35,992 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED


%6|1734896174.362|FAIL|rdkafka#producer-3| [thrd:ssl://51.161.81.188:9093/bootstrap]: ssl://51.161.81.188:9093/1: Disconnected (after 98635ms in state UP)
%6|1734896226.853|FAIL|rdkafka#producer-3| [thrd:ssl://51.161.81.188:9093/bootstrap]: ssl://51.161.81.188:9093/1: Disconnected (after 49675ms in state UP, 1 identical error(s) suppressed)
%6|1734896228.506|FAIL|rdkafka#producer-3| [thrd:ssl://51.161.81.208:9093/bootstrap]: ssl://51.161.81.208:9093/2: Disconnected (after 50044ms in state UP, 1 identical error(s) suppressed)


2024-12-22 20:37:12,200 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2024-12-22 20:37:12,369 INFO: Waiting for log aggregation to finish.
2024-12-22 20:37:28,348 INFO: Execution finished successfully.


2024-12-22 20:37:34.094 | INFO     | __main__:<module>:5 - ✅ Uploaded 'transactions' Feature Group to Hopsworks!


#### Interactions

In [46]:
logger.info("Uploading 'interactions' Feature Group to Hopsworks.")
interactions_fg = feature_store.create_interactions_feature_group(
    fs=fs, df=interaction_df, online_enabled=True
)
logger.info("✅ Uploaded 'interactions' Feature Group to Hopsworks!!")

2024-12-22 20:37:34.144 | INFO     | __main__:<module>:1 - Uploading 'interactions' Feature Group to Hopsworks.
%6|1734896278.645|FAIL|rdkafka#producer-3| [thrd:ssl://51.161.80.189:9093/bootstrap]: ssl://51.161.80.189:9093/0: Disconnected (after 100263ms in state UP, 1 identical error(s) suppressed)
%6|1734896330.771|FAIL|rdkafka#producer-3| [thrd:ssl://51.161.81.208:9093/bootstrap]: ssl://51.161.81.208:9093/2: Disconnected (after 50000ms in state UP, 1 identical error(s) suppressed)
%6|1734896331.631|FAIL|rdkafka#producer-3| [thrd:ssl://51.161.81.188:9093/bootstrap]: ssl://51.161.81.188:9093/1: Disconnected (after 49300ms in state UP, 1 identical error(s) suppressed)
Uploading Dataframe: 100.00% |██████████| Rows 93520/93520 | Elapsed Time: 01:20 | Remaining Time: 00:00


Launching job: interactions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1197208/jobs/named/interactions_1_offline_fg_materialization/executions
2024-12-22 20:39:10,958 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2024-12-22 20:39:14,267 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED


%6|1734896415.268|FAIL|rdkafka#producer-3| [thrd:ssl://51.161.80.189:9093/bootstrap]: ssl://51.161.80.189:9093/0: Disconnected (after 132448ms in state UP, 1 identical error(s) suppressed)


2024-12-22 20:41:06,231 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2024-12-22 20:41:06,398 INFO: Waiting for log aggregation to finish.


%6|1734896467.491|FAIL|rdkafka#producer-3| [thrd:ssl://51.161.81.208:9093/bootstrap]: ssl://51.161.81.208:9093/2: Disconnected (after 49755ms in state UP, 1 identical error(s) suppressed)
%6|1734896468.313|FAIL|rdkafka#producer-3| [thrd:ssl://51.161.81.188:9093/bootstrap]: ssl://51.161.81.188:9093/1: Disconnected (after 50001ms in state UP, 1 identical error(s) suppressed)


2024-12-22 20:41:28,655 INFO: Execution finished successfully.


2024-12-22 20:41:32.492 | INFO     | __main__:<module>:5 - ✅ Uploaded 'interactions' Feature Group to Hopsworks!!


In [48]:
# import hopsworks 
# project = hopsworks.login(api_key_value="H")
# fs = project.get_feature_store(name='id2223artsy_featurestore')
artworks_fg = fs.get_feature_group('artworks', version=1)

ERROR! Session/line number was not unique in database. History logging moved to new session 71


# Compute ranking dataset

The last step is to compute the ranking dataset used to train the scoring/ranking model from the feature groups we've just created:


In [ ]:
# ranking_df = compute_ranking_dataset(
#     trans_fg,
#     artworks_fg,
#     users_fg,
# )
# ranking_df.shape

The ranking dataset looks as follows:

In [41]:
ranking_df = compute_ranking_dataset2(
    transactions_df,
    artworks_df,
    users_df,
)
ranking_df.shape

(61666, 7)

In [42]:
ranking_df.head(3)

user_id,age,artwork_id,label,title,description,category
str,f64,str,i32,str,str,str
"""a0ed6728-dc2d-4fb8-b0dc-f6a32e…",28.0,"""516cbff8fdc441ac4400017e""",1,"""Old Man with a Beard""","""The image features deep, dark …","""Print"""
"""a0ed6728-dc2d-4fb8-b0dc-f6a32e…",28.0,"""515bc80a94714c1d2e001dd3""",1,"""Francesco Goya y Lucientes, Pi…","""The image features a muted col…","""Print"""
"""a0ed6728-dc2d-4fb8-b0dc-f6a32e…",28.0,"""515b34d538ad2dfb2f000605""",1,"""Anfiteatro Flavio detto il Col…","""The etching presents a monochr…","""Print"""


In [43]:
ranking_df.get_column("label").value_counts()

label,count
i32,u32
0,56060
1,5606


In [50]:
logger.info("Uploading 'ranking' Feature Group to Hopsworks.")
rank_fg = feature_store.create_ranking_feature_group(
    fs,
    df=ranking_df,
    parents=[artworks_fg, users_fg, trans_fg],
    online_enabled=False
)
logger.info("✅ Uploaded 'ranking' Feature Group to Hopsworks!!")

2024-12-22 20:48:26.743 | INFO     | __main__:<module>:1 - Uploading 'ranking' Feature Group to Hopsworks.


Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1197208/fs/1186887/fg/1393125






%6|1734896932.296|FAIL|rdkafka#producer-3| [thrd:ssl://51.161.81.188:9093/bootstrap]: ssl://51.161.81.188:9093/1: Disconnected (after 97599ms in state UP)



%6|1734896943.193|FAIL|rdkafka#producer-9| [thrd:ssl://51.161.81.188:9093/bootstrap]: ssl://51.161.81.188:9093/1: Disconnected (after 99812ms in state UP)








%6|1734896984.538|FAIL|rdkafka#producer-3| [thrd:ssl://51.161.81.208:9093/bootstrap]: ssl://51.161.81.208:9093/2: Disconnected (after 49700ms in state UP, 1 identical error(s) suppressed)
%6|1734896985.710|FAIL|rdkafka#producer-3| [thrd:ssl://51.161.80.189:9093/bootstrap]: ssl://51.161.80.189:9093/0: Disconnected (after 49438ms in state UP, 1 identical error(s) suppressed)


%6|1734896995.383|FAIL|rdkafka#producer-9| [thrd:ssl://51.161.81.188:9093/bootstrap]: ssl://51.161.81.188:9093/1: Disconnected (after 49997ms in state UP, 1 identical error(s) suppressed)
%6|1734896995.730|FAIL|rdkafka#producer-9| [thrd:ssl://51.161.80.189:9093/bootstrap]: ssl://51.161.80.189:909

Launching job: ranking_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1197208/jobs/named/ranking_1_offline_fg_materialization/executions


%6|1734897138.889|FAIL|rdkafka#producer-3| [thrd:ssl://51.161.80.189:9093/bootstrap]: ssl://51.161.80.189:9093/0: Disconnected (after 98849ms in state UP, 1 identical error(s) suppressed)


2024-12-22 20:52:19,439 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2024-12-22 20:52:22,611 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED


%6|1734897177.813|FAIL|rdkafka#producer-9| [thrd:ssl://51.161.81.208:9093/bootstrap]: ssl://51.161.81.208:9093/2: Disconnected (after 127537ms in state UP, 1 identical error(s) suppressed)
%6|1734897191.529|FAIL|rdkafka#producer-3| [thrd:ssl://51.161.81.188:9093/bootstrap]: ssl://51.161.81.188:9093/1: Disconnected (after 49997ms in state UP, 1 identical error(s) suppressed)
%6|1734897191.549|FAIL|rdkafka#producer-3| [thrd:ssl://51.161.81.208:9093/bootstrap]: ssl://51.161.81.208:9093/2: Disconnected (after 49999ms in state UP, 1 identical error(s) suppressed)
%6|1734897229.581|FAIL|rdkafka#producer-9| [thrd:ssl://51.161.80.189:9093/bootstrap]: ssl://51.161.80.189:9093/0: Disconnected (after 50001ms in state UP, 1 identical error(s) suppressed)
%6|1734897229.947|FAIL|rdkafka#producer-9| [thrd:ssl://51.161.81.188:9093/bootstrap]: ssl://51.161.81.188:9093/1: Disconnected (after 50000ms in state UP, 1 identical error(s) suppressed)
%6|1734897242.442|FAIL|rdkafka#producer-3| [thrd:ssl://51.1

2024-12-22 20:54:04,716 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2024-12-22 20:54:04,872 INFO: Waiting for log aggregation to finish.
2024-12-22 20:54:20,172 INFO: Execution finished successfully.


KeyError: "'FeatureGroup' object has no feature called 'customer_id'."

%6|1734897280.134|FAIL|rdkafka#producer-9| [thrd:ssl://51.161.81.208:9093/bootstrap]: ssl://51.161.81.208:9093/2: Disconnected (after 98917ms in state UP, 1 identical error(s) suppressed)
%6|1734897294.200|FAIL|rdkafka#producer-3| [thrd:ssl://51.161.81.188:9093/bootstrap]: ssl://51.161.81.188:9093/1: Disconnected (after 50001ms in state UP, 1 identical error(s) suppressed)
%6|1734897295.089|FAIL|rdkafka#producer-3| [thrd:ssl://51.161.80.189:9093/bootstrap]: ssl://51.161.80.189:9093/0: Disconnected (after 50044ms in state UP, 1 identical error(s) suppressed)
%6|1734897332.563|FAIL|rdkafka#producer-9| [thrd:ssl://51.161.81.188:9093/bootstrap]: ssl://51.161.81.188:9093/1: Disconnected (after 50066ms in state UP, 1 identical error(s) suppressed)
%6|1734897332.779|FAIL|rdkafka#producer-9| [thrd:ssl://51.161.80.189:9093/bootstrap]: ssl://51.161.80.189:9093/0: Disconnected (after 49757ms in state UP, 1 identical error(s) suppressed)
%6|1734897345.363|FAIL|rdkafka#producer-3| [thrd:ssl://51.16